In [1]:
# using the method according to jinjin's paper
import pandas as pd
import numpy as np
from scipy.stats import lognorm

In [2]:
df = pd.read_csv('C:/Users/mchen48/Box/01 Research/PFASs/PFASs_in_Carpet/00 PFAS_US_carpet/PFAS_flows_stocks_data_processing/Basic_data.csv')

In [3]:
# rao input parameters such as the concentration of different PFAS at different scenarios
# the unit of all V parameters are million m2
P_import = 664     # change according to different type of PFAS, ug/m2
P_production  = 88200  # change according to different type of PFAS, ug/m2
P_export = P_production   # change according to different type of PFAS, ug/m2
R_to_carpet = 0.36   # the ratio of collected carpet recycled to carpet
P_yearlyloss_air = 0  # change according to different type of PFAS, ug/m2
P_yearlyloss_dust = 1.752     # change according to different type of PFAS, ug/m2
Prod_emi_R = 0.15 # High percentage loss of PFAS during production, related to F9, the high is 0.15, the low is 0.05
P_protector = 0.375 # kg/million m2 parameter to calculate the PFAS use through carpet protector

In [4]:
# creat a lognormal distribution
age_iniStock = 4.3  # the initial age of nylon carpet stock (in 1990) is 4.3 years old
shape = 0.2957397   # standard deviation
scale = np.exp(2.1080312) # mean
loc = 0
dist = lognorm(shape, loc, scale)

In [5]:
# calculate In-Use Stock of PFAS in nylon carpet
Af = 159 # the average single family house size before 1990 is 159 m2
P_cf = 0.51
S0_volumn = (Af/3.35) * P_cf * df.loc[0,'us_pop'] * df.loc[0,'Percent_nylon'] # Calculate the initial carpet stock volumn in use (initial average family size 3.35)
S0_PFAS  = (S0_volumn * P_production * 0.85)/1000  # Calculate the initial PFAS stock in use

In [6]:
# creat the column of F1_SP_High
df['F1'] = 0 ## the unit of F1_SP_High is kg

In [7]:
# calculate F1: PFAS flow in imported nylon carpet
for i in range(len(df)):
    df.loc[i,'F1'] = (df.loc[i,'V_import'] * P_import * df.loc[i,'Percent_nylon'] - 
                              df.loc[i,'V_export'] * P_export * df.loc[i,'Percent_nylon'])/1000

In [8]:
# creat the column of F_production
df['F_production'] = 0 ## the unit of F_production is kg

In [9]:
# calculate F_production:PFAS flow used in the domestic carpet production in nylon carpet
for i in range(len(df)):
    df.loc[i,'F_production'] = (df.loc[i,'V_production'] * P_production * df.loc[i,'Percent_nylon']*df.loc[i,'PhaseOut'])/1000

In [10]:
df['F3'] = 0 

In [11]:
# calculate F3:PFAS flow in material loss in production
for i in range(len(df)):
    df.loc[i,'F3'] = df.loc[i,'F_production'] * 0.12 * 0.15

In [12]:
df_yearly_EoL = pd.DataFrame() #creat an empty dataframe to store every year's EoL carpet according to lognormal distribution

In [13]:
for year in range(1990,2061):
    df_yearly_EoL[year] = 0

In [14]:
for i in range(0,71):
    df_yearly_EoL.loc[i,:] = 0

In [15]:
for i in range(0,71):
    for year in range(1990,2061):
        if year >= df.loc[i,'year']:
            df_yearly_EoL.loc[i,year] = (dist.cdf(year - df.loc[i,'year'] + 1) - dist.cdf(year - df.loc[i,'year'])) * df.loc[i,'V_sales_nylon']
        else:
            df_yearly_EoL.loc[i,year] = df_yearly_EoL.loc[i,year]

In [16]:
# creat a column to store the EoL of initial stock, lognormal distribution, initial age is 4.3 years
# mean and std are the same as yearly EoL of sales volumn
df['V_iniStock_EoL'] = 0

In [17]:
for i in range(0,71):
    df.loc[i,'V_iniStock_EoL'] = ((dist.cdf(age_iniStock+i+1) - dist.cdf(age_iniStock+i))/
                                  (1-dist.cdf(age_iniStock)))*S0_volumn       

In [18]:
# generate a column to store the yearly EoL from every year's new sales to the market
df['V_new_EoL'] = 0

In [19]:
# calculate the yearly end of life carpet volumn
b = 0
for year in range(1990, 2061):
    a = 0
    for i in range(0,71):
        a = a + df_yearly_EoL.loc[i,year]
    df.loc[b,'V_new_EoL'] = a
    b = b + 1

### Calculate v4_phaseout, which means under the phaseout scenario

In [20]:
# generate a column to store the yearly EoL from every year's new sales to the market in phase out scenario
df['V_new_EoL_PhaseOut'] = 0

In [21]:
b = 0
for year in range(1990, 2061):
    a = 0
    for i in range(0,71):
        a = a + df_yearly_EoL.loc[i,year]*df.loc[i,'PhaseOut']
    df.loc[b,'V_new_EoL_PhaseOut'] = a
    b = b + 1

In [22]:
# generate a column to store the total EoL volumn from stock carpet and every year's new carpet
df['V4_PhaseOut'] = 0

In [23]:
for i in range(0,71):
    df.loc[i,'V4_PhaseOut'] = df.loc[i,'V_iniStock_EoL'] + df.loc[i,'V_new_EoL_PhaseOut']

In [24]:
df['V_delta_S_PhaseOut'] = 0 # add a column to stock yearly nylon carpet volumn stock change under phaseout scenario

In [25]:
# calculate the yearly change of nylon carpet in use stock under phaseout scenario
for i in range(0,71):
     df.loc[i,'V_delta_S_PhaseOut'] = df.loc[i,'V_sales_nylon']*df.loc[i,'PhaseOut'] - df.loc[i,'V4_PhaseOut']

In [26]:
df['V_stock_PhaseOut'] = 0 # add a column to stock yearly total nylon carpet volumn stock 

In [27]:
# calculate the total carpet in stock every year
df.loc[0,'V_stock_PhaseOut'] = S0_volumn + df.loc[0,'V_delta_S_PhaseOut']
for i in range(1,71):
     df.loc[i,'V_stock_PhaseOut'] = df.loc[i-1,'V_stock_PhaseOut'] + df.loc[i, 'V_delta_S_PhaseOut']

#### calculate normal v4

In [28]:
# generate a column to store the total EoL volumn from stock carpet and every year's new carpet
df['V4'] = 0

In [29]:
for i in range(0,71):
    df.loc[i,'V4'] = df.loc[i,'V_iniStock_EoL'] + df.loc[i,'V_new_EoL']

In [30]:
df['V_delta_S'] = 0 # add a column to stock yearly nylon carpet volumn stock change

In [31]:
# calculate the yearly change of nylon carpet in use stock
for i in range(0,71):
     df.loc[i,'V_delta_S'] = df.loc[i,'V_sales_nylon'] - df.loc[i,'V4']

In [32]:
df['V_stock'] = 0 # add a column to stock yearly total nylon carpet volumn stock 

In [33]:
# calculate the total carpet in stock every year
df.loc[0,'V_stock'] = S0_volumn + df.loc[0,'V_delta_S']
for i in range(1,71):
     df.loc[i,'V_stock'] = df.loc[i-1,'V_stock'] + df.loc[i, 'V_delta_S']

In [34]:
df['F_to_air'] = 0 # add a column to stock yearly in use PFAS lost to air

In [35]:
df['F_to_dust'] = 0 # add a column to stock yearly in use PFAS lost to dust

In [36]:
df['F6'] = 0 # add a column to stock yearly PFAS emission from in use carpet

In [37]:
# calculate the PFAS emissions from in-use stock
for i in range(0,71):
    df.loc[i,'F_to_air'] = P_yearlyloss_air * df.loc[i,'V_stock_PhaseOut']/1000
    df.loc[i,'F_to_dust'] = P_yearlyloss_dust * df.loc[i,'V_stock_PhaseOut']/1000
    df.loc[i,'F6'] = (df.loc[i,'F_to_air'] + df.loc[i,'F_to_dust'])*2

### Calculation of PFAS from the protectors used every year (F10)

In [38]:
df_yearly_left = pd.DataFrame() #creat an empty dataframe to store every year's EoL carpet according to lognormal distribution
for year in range(1990,2061):
    df_yearly_left[year] = 0
for i in range(0,71):
    df_yearly_left.loc[i,:] = 0

In [39]:
for i in range(0,71):
    for year in range(1990,2061):
        if year >= df.loc[i,'year']:
            sum_PreYear = 0
            for j in range(df.loc[i,'year'], year+1):
                sum_PreYear = sum_PreYear + df_yearly_EoL.loc[i,j]
            df_yearly_left.loc[i,year] = df.loc[i,'V_sales_nylon'] - sum_PreYear 
        else:
            df_yearly_left.loc[i,year] = df_yearly_left.loc[i,year]

In [40]:
df['V_new_left'] = 0

In [41]:
# calculate the yearly left in stock from new producted carpet
b = 4
for year in range(1994, 2061):
    a = 0
    if year%2==0:
        for i in range(0,71,2):
            a = a + df_yearly_left.loc[i,year]
    else:
        for i in range(1,71,2):
            a = a + df_yearly_left.loc[i,year]
    df.loc[b,'V_new_left'] = a
    b = b + 1

In [42]:
# creat a column to store the PFAS use for in use carpet protector
df['F10'] = 0

In [43]:
# calculate the yearly PFAS used for in use protector
for i in range(0,71):
    df.loc[i,'F10'] = df.loc[i,'V_new_left'] * P_protector

### In_use stock calculation

In [44]:
df['F_delta_S'] = 0 # add a column to stock yearly total PFAS change in stock

In [45]:
df['F_stock'] = 0 # add a column to stock yearly total PFAS in stock

In [46]:
df['F2'] = 0
df['F4'] = 0
df['F5'] = 0

In [47]:
#method 2: using the method in excel template, according to the potion of in use carpet to calculate
df.loc[0,'F4'] = (df.loc[0,'V4']/S0_volumn)*S0_PFAS
df.loc[0,'F2'] = df.loc[0,'F1'] + df.loc[0,'F_production'] + df.loc[0,'F5']- df.loc[0,'F3']
df.loc[0,'F_delta_S'] = df.loc[0,'F2'] + df.loc[0,'F10'] - df.loc[0,'F4'] - df.loc[0,'F6']
df.loc[0,'F_stock'] = df.loc[0,'F_delta_S'] + S0_PFAS
for i in range(1,71):
    df.loc[i,'F4'] = (df.loc[0,'V4']/df.loc[i-1,'V_stock'])*df.loc[i-1,'F_stock']
    df.loc[i,'F5'] = df.loc[i-1,'F4'] * df.loc[i,'RR_total'] * R_to_carpet
    df.loc[i,'F2'] = df.loc[i,'F1'] + df.loc[i,'F_production'] + df.loc[i,'F5']- df.loc[i,'F3']
    df.loc[i,'F_delta_S'] = df.loc[i,'F2'] + df.loc[i,'F10'] - df.loc[i,'F4'] - df.loc[i,'F6']
    df.loc[i,'F_stock'] = df.loc[i,'F_delta_S'] + df.loc[i-1,'F_stock']    

In [48]:
df['F8'] = 0

In [49]:
# calculate the PFAS flow in discarded carpet to incineration
for i in range(0,71):
    df.loc[i,'F8'] = df.loc[i,'F4'] * df.loc[i,'IncR']

In [50]:
df['F9'] = 0

In [51]:
# calculate the PFAS emission during carpet production
for i in range(0,71):
    df.loc[i,'F9'] = df.loc[i,'F_production'] * Prod_emi_R

In [52]:
df['F_EoL_total'] = 0

In [53]:
# calculate the yearly total EoL, including F4 and F3
for i in range(0,71):
    df.loc[i,'F_EoL_total'] = df.loc[i,'F3'] + df.loc[i,'F4']

In [54]:
df['landfill_ratio'] = 0 # the ratio is calculated using 1 - recycle ratio - incineration ratio

In [55]:
for i in range(0,71):
    df.loc[i,'landfill_ratio'] = 1 - df.loc[i,'RR_total'] - df.loc[i,'IncR']

In [56]:
df['F_landfill_yearly'] = 0 # store every year's landfill PFAS

In [57]:
# calculate the yearly landfill PFAS
for i in range(0,71):
    df.loc[i,'F_landfill_yearly'] = df.loc[i,'F_EoL_total'] * df.loc[i,'landfill_ratio']

In [58]:
df['F_accu_landfill'] = 0 # store accumulated landfill PFAS, in unit of tonnes

In [59]:
# calculate the accumulated PFAS to landfill for every year
for i in range(0,71):
    a = 0
    for j in range(0,i+1):
        a = a + df.loc[j,'F_landfill_yearly']
    df.loc[i,'F_accu_landfill'] = a

In [60]:
df.to_csv('SP_HIGH_PhaseOut_v2.csv',index = False)